## Assignment1 COMP90049

'''

Date created: 03/27/2018

Date last modified: 04/11/2018

Python Version: 2.7

'''

##### Accuracy, Precision and Recall

The following code implements three functions, each for calculating accuracy, precision and recall

In [18]:
def calculate_accuracy(predicted_list, correct_list):
    i = 0
    count = 0
    while(i < len(predicted_list)):
        if predicted_list[i] == correct_list[i]:
            count += 1
        i += 1
    return count / float(len(predicted_list))

def calculate_precision(predicted_list, correct_list):
    i = 0
    count = 0
    total = 0

    while(i < len(predicted_list)):
        if correct_list[i] in predicted_list[i]:
            count += 1
        total += len(predicted_list[i])
        i += 1
    return count / float(total)

def calculate_recall(predicted_list, correct_list):
    i = 0
    count = 0
    while(i < len(predicted_list)):
        if correct_list[i] in predicted_list[i]:
            count += 1
        i += 1

    return count / float(len(correct_list))

#### Global Edit Distance
In the following code global edit distance search is implemented by calculating Levenshtein Distance. A list of sets containing predicted words is retrieved "predicted_list".

In [30]:
import editdistance

import time
start = time.time()

dictionary_set = set(record.strip() for record in open('dictionary.txt'))
misspell_list = []
correct_list = []
predicted_list = []

with open("correct.txt") as record :
    for line in record:
        word = line.strip()
        correct_list.append(word)
        
with open("misspell.txt") as record :
    for line in record:
        word = line.strip()
        misspell_list.append(word)
        if word in dictionary_set :
            predicted_list.append(set([word]))
        else:  
            minimum = float("inf")
            predicted_words = set()
            for word_dict in dictionary_set:
                temp_stor = editdistance.eval(word, word_dict)
                if temp_stor < minimum :
                    predicted_words.clear()
                    predicted_words.add(word_dict)
                    minimum = temp_stor
                elif temp_stor == minimum :
                    predicted_words.add(word_dict)
            predicted_list.append(predicted_words)
end = time.time()
time_spent = (end - start)

###### GED - Levenshtein Distance (Precision and Recall Score)

In [31]:
print "------------------"
print "The precision-score of the method used is"
print calculate_precision(predicted_list, correct_list)
print "------------------"
print "The recall-score of the method used is"
print calculate_recall(predicted_list, correct_list)
print "------------------"
print "The time spent in execution of code is"
print time_spent

------------------
The precision-score of the method used is
0.0457670043415
------------------
The recall-score of the method used is
0.353351955307
------------------
The time spent in execution of code is
314.933781862


#### Neighbourhood Search

In [8]:

def neighbour_generator(word, k):
    predicted_words = set()
    for word_dict in dictionary_set:
        temp_stor = editdistance.eval(word, word_dict)
        if temp_stor <= k :
            predicted_words.add(word_dict)
    return predicted_words
    
predicted_list_k1 = []
    
for word in misspell_list:
    predicted_words = neighbour_generator(word, 1)
    predicted_list_k1.append(predicted_words)
    
predicted_list_k2 = []
    
for word in misspell_list:
    predicted_words = neighbour_generator(word, 2)
    predicted_list_k2.append(predicted_words)

predicted_list_k3 = []
    
for word in misspell_list:
    predicted_words = neighbour_generator(word, 3)
    predicted_list_k3.append(predicted_words)
    
predicted_list_k4 = []
    
for word in misspell_list:
    predicted_words = neighbour_generator(word, 4)
    predicted_list_k4.append(predicted_words)

##### The precision and recall score based of different values of k in neighbourhood search method

In [20]:
print "------------------"
print "The precision-score of the method when k = 1 is"
print calculate_precision(predicted_list_k1, correct_list)
print "------------------"
print "The recall-score of the method when k = 1 is"
print calculate_recall(predicted_list_k1, correct_list)
print "------------------"
print "------------------"
print "The precision-score of the method when k = 2 is"
print calculate_precision(predicted_list_k2, correct_list)
print "------------------"
print "The recall-score of the method when k = 2 is"
print calculate_recall(predicted_list_k2, correct_list)
print "------------------"
print "------------------"
print "The precision-score of the method when k = 3 is"
print calculate_precision(predicted_list_k3, correct_list)
print "------------------"
print "The recall-score of the method when k = 3 is"
print calculate_recall(predicted_list_k3, correct_list)
print "------------------"
print "------------------"
print "The precision-score of the method when k = 4 is"
print calculate_precision(predicted_list_k4, correct_list)
print "------------------"
print "The recall-score of the method when k = 4 is"
print calculate_recall(predicted_list_k4, correct_list)

------------------
The precision-score of the method when k = 1 is
0.0438121047877
------------------
The recall-score of the method when k = 1 is
0.406424581006
------------------
------------------
The precision-score of the method when k = 2 is
0.00353290408897
------------------
The recall-score of the method when k = 2 is
0.708100558659
------------------
------------------
The precision-score of the method when k = 3 is
0.000349547267871
------------------
The recall-score of the method when k = 3 is
0.784916201117
------------------
------------------
The precision-score of the method when k = 4 is
6.54787652589e-05
------------------
The recall-score of the method when k = 4 is
0.815642458101


##### N-Gram Distance Method

The following function implements n-gram prediction method. It takes a misspelled word and N-value as argument and returns a set of predicted words,  

In [5]:
import ngram
from multiprocessing import Pool

def ngram_predict(*args):
    word = args[0][0]
    n_val = args[0][1]
    predicted_words = set()
    maximum = -1
    for word_dict in dictionary_set:
        temp_stor = ngram.NGram.compare(word, word_dict, N = n_val)
        if temp_stor > maximum :
            predicted_words.clear()
            predicted_words.add(word_dict)
            maximum = temp_stor
        elif temp_stor == maximum :
            predicted_words.add(word_dict)
    return predicted_words

##### 2-gram

In [15]:
'''
Created on 26 Jun. 2018

@author: Anubhav Singh
'''
import sys
import json
import os.path
from multiprocessing import Pool
from functools import partial
from multiprocessing import cpu_count
import copyreg
import types
import stanfordcorenlp

def _pickle_method(method):
    func_name = method.im_func.__name__
    obj = method.im_self
    cls = method.im_class
    if func_name.startswith('__') and not func_name.endswith('__'): #deal with mangled names
        cls_name = cls.__name__.lstrip('_')
        func_name = '_' + cls_name + func_name
    return _unpickle_method, (func_name, obj, cls)

def _unpickle_method(func_name, obj, cls):
    for cls in cls.__mro__:
        try:
            func = cls.__dict__[func_name]
        except KeyError:
            pass
        else:
            break
    return func.__get__(obj, cls)


copyreg.pickle(types.MethodType, _pickle_method, _unpickle_method)

#===============================================================================
# 
#===============================================================================
class Documents(object):
    '''
    classdocs
    '''

    def __init__(self,corenlp=None):
        '''
        Constructor
        '''

        self.corenlp = corenlp
        #------------------------------------------------ Load the dataset files
        with open('/home/xoxo/eclipse-workspace/zUra/dataset/documents.json') as f:
            self.documents_json = json.load(f)

#===============================================================================
# 
#===============================================================================
    def generate_tagged_corpus_stanfordnlp(self):
        p = Pool(4)
        self.documents_json = p.map(self.tokenize_n_tag_stanfordnlp, [1,2,3])

#===============================================================================
# 
#===============================================================================
    def tokenize_n_tag_stanfordnlp(self, doc):
        print(doc)



corenlp = stanfordcorenlp.StanfordCoreNLP(r'/home/xoxo/eclipse-workspace/zUra/src/stanford-corenlp-full-2018-02-27')
document_obj = Documents(corenlp)
document_obj.generate_tagged_corpus_stanfordnlp()

TypeError: can't pickle _thread.lock objects

##### N-gram with n = 2, Precision and Recall Score 

In [2]:
print "------------------"
print "The precision-score of the method used is"
print calculate_precision(predicted_list_ngram2, correct_list)
print "------------------"
print "The recall-score of the method used is"
print calculate_recall(predicted_list_ngram2, correct_list)

SyntaxError: Missing parentheses in call to 'print'. Did you mean print("------------------")? (<ipython-input-2-faaeca3b46b4>, line 1)

##### 3-gram

In [22]:
p = Pool(4)

args_list = []
for word in misspell_list:
    args_list.append((word,3))
predicted_list_ngram3 = p.map(ngram_predict,args_list)

##### N-gram with n = 3, Precision and Recall Score 

In [24]:
print "------------------"
print "The precision-score of the method used is"
print calculate_precision(predicted_list_ngram3, correct_list)
print "------------------"
print "The recall-score of the method used is"
print calculate_recall(predicted_list_ngram3, correct_list)

------------------
The precision-score of the method used is
0.126829268293
------------------
The recall-score of the method used is
0.18156424581
